<a href="https://colab.research.google.com/github/ealbarracint-dev/Parcial2Se-ales/blob/main/Parcial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PUNTO 1

In [ ]:
!pip install yt-dlp pydub soundfile matplotlib numpy

import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
from pydub import AudioSegment
import yt_dlp
import os

# ----------------------------------------------------
# 1. DESCARGA AUTOMÁTICA (5 SEGUNDOS FIJOS)
# ----------------------------------------------------

YOUTUBE_URL = "https://youtu.be/ErCAOMi5EGM?si=PAAR4NhkQiPuIjBS"

def download_fixed_youtube_audio(output_file="message.wav", duration_sec=5):
    """
    Descarga audio desde YouTube.
    Extrae exactamente duration_sec segundos.
    """
    print("Descargando audio desde YouTube...")

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'temp_audio.%(ext)s',
        'quiet': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([YOUTUBE_URL])

    # Encuentra archivo descargado
    for f in os.listdir():
        if f.startswith("temp_audio"):
            downloaded_file = f
            break

    print("Convirtiendo a WAV...")

    audio = AudioSegment.from_file(downloaded_file)
    audio = audio.set_channels(1)       # Mono
    audio = audio.set_frame_rate(44100) # fs = 44100 Hz
    audio = audio[:duration_sec * 1000] # EXACTAMENTE 5 segundos

    audio.export(output_file, format="wav")
    os.remove(downloaded_file)

    print("Audio listo como", output_file)


# ----------------------------------------------------
# 2. FFT Utilities
# ----------------------------------------------------

def fft_mag(x, fs):
    N = len(x)
    X = np.fft.fftshift(np.fft.fft(x))
    f = np.fft.fftshift(np.fft.fftfreq(N, 1/fs))
    return f, np.abs(X) / N

def apply_lpf_fft(x, fs, fcut):
    N = len(x)
    X = np.fft.fft(x)
    freqs = np.fft.fftfreq(N, 1/fs)
    mask = np.abs(freqs) <= fcut
    X_filtered = X * mask
    return np.real(np.fft.ifft(X_filtered))


# ----------------------------------------------------
# 3. MODULACIÓN + DEMODULACIÓN
# ----------------------------------------------------

def simulate_dsb_demod(audio_file="message.wav", A1=1.0, f0=15000, fcut=5000):

    m, fs = sf.read(audio_file)
    if m.ndim > 1:
        m = m[:,0]
    N = len(m)
    t = np.arange(N) / fs

    # Portadora
    carrier = np.cos(2*np.pi*f0*t)

    # (1) Modulación DSB-SC
    x = A1 * m * carrier

    # (2) Mezclador
    lo = np.cos(2*np.pi*f0*t)
    y = x * lo

    # (3) LPF usando FFT
    y_lpf = apply_lpf_fft(y, fs, fcut)

    # (4) Escalado
    recovered = (2/A1) * y_lpf

    # FFTs
    f_m, Mmag = fft_mag(m, fs)
    f_x, Xmag = fft_mag(x, fs)
    f_y, Ymag = fft_mag(y, fs)
    f_yl, Ylmag = fft_mag(y_lpf, fs)
    f_r, Rmag = fft_mag(recovered, fs)

    # ----------------------------------------------------
    # Gráficas en tiempo
    # ----------------------------------------------------
    plt.figure(figsize=(12,10))
    plt.subplot(5,1,1); plt.plot(t, m); plt.title("m(t) — mensaje")
    plt.subplot(5,1,2); plt.plot(t, x); plt.title("x(t) — señal modulada")
    plt.subplot(5,1,3); plt.plot(t, y); plt.title("y(t) — salida del mezclador")
    plt.subplot(5,1,4); plt.plot(t, y_lpf); plt.title("y_LPF(t) — salida del LPF (FFT)")
    plt.subplot(5,1,5); plt.plot(t, recovered); plt.title("Señal recuperada m_rec(t)")
    plt.tight_layout()
    plt.show()

    # ----------------------------------------------------
    # Gráficas en frecuencia
    # ----------------------------------------------------
    plt.figure(figsize=(12,10))
    plt.subplot(5,1,1); plt.plot(f_m, Mmag); plt.title("M(f) — espectro del mensaje")
    plt.subplot(5,1,2); plt.plot(f_x, Xmag); plt.title("X(f) — espectro modulada")
    plt.subplot(5,1,3); plt.plot(f_y, Ymag); plt.title("Y(f) — post-mezclador")
    plt.subplot(5,1,4); plt.plot(f_yl, Ylmag); plt.title("Y_LPF(f)")
    plt.subplot(5,1,5); plt.plot(f_r, Rmag); plt.title("M_rec(f)")
    plt.tight_layout()
    plt.show()


# ----------------------------------------------------
# 4. EJECUCIÓN AUTOMÁTICA
# ----------------------------------------------------

if __name__ == "__main__":
    download_fixed_youtube_audio()
    simulate_dsb_demod()

PUNTO 2

In [ ]:

# ====================================================
#     PARCIAL 2 - PUNTO 2
#     Sistema Masa–Resorte–Amortiguador
#     Gráficas: Polos, Bode, Impulso, Escalón, Rampa
# ====================================================

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# ----------------------------------------------------
# Función para graficar el caso dinámico
# ----------------------------------------------------
def plot_case(name, m, c, k, color='blue'):
    num = [1]
    den = [m, c, k]

    # Sistema mecánico X(s)/F(s)
    sys = signal.TransferFunction(num, den)

    # Lazo cerrado con realimentación unitaria
    num_cl = num
    den_cl = np.polyadd(den, num)   # H_cl = H / (1 + H)
    sys_cl = signal.TransferFunction(num_cl, den_cl)

    # -------------------------------------------
    # Cálculo dinámico
    wn = np.sqrt(k/m)
    zeta = c / (2*np.sqrt(m*k))
    if zeta < 1:
        wd = wn*np.sqrt(1 - zeta**2)
        tp = np.pi / wd
        ts = 4 / (zeta * wn)
        tr = 1.8 / wn
    else:
        wd = None
        tp = None
        ts = 4 / (zeta * wn)
        tr = None

    # -------------------------------------------
    # PLOT GENERAL
    t = np.linspace(0, 10, 1000)
    fig, axs = plt.subplots(3, 2, figsize=(12, 14))

    # --- 1. Polos y ceros ---
    z, p, k_tf = signal.tf2zpk(num, den)
    axs[0, 0].scatter(np.real(p), np.imag(p), c=color, s=60, label='Polos')
    axs[0, 0].axhline(0, color='black', linewidth=0.8)
    axs[0, 0].axvline(0, color='black', linewidth=0.8)
    axs[0, 0].grid(True)
    axs[0, 0].set_title(f"Polos del sistema - {name}")
    axs[0, 0].set_xlabel("Real")
    axs[0, 0].set_ylabel("Imaginario")

    # --- 2. Bode magnitud ---
    w, mag, phase = signal.bode(sys)
    axs[0, 1].semilogx(w, mag, color=color)
    axs[0, 1].set_title("Diagrama de Bode - Magnitud")
    axs[0, 1].set_xlabel("Frecuencia (rad/s)")
    axs[0, 1].set_ylabel("Magnitud (dB)")
    axs[0, 1].grid(True)

    # --- 3. Impulso ---
    t_imp, y_imp = signal.impulse(sys, T=t)
    axs[1, 0].plot(t_imp, y_imp, color=color)
    axs[1, 0].set_title("Respuesta al impulso")
    axs[1, 0].set_xlabel("Tiempo (s)")
    axs[1, 0].grid(True)

    # --- 4. Escalón ---
    t_step, y_step = signal.step(sys, T=t)
    axs[1, 1].plot(t_step, y_step, color=color)
    axs[1, 1].set_title("Respuesta al escalón")
    axs[1, 1].set_xlabel("Tiempo (s)")
    axs[1, 1].grid(True)

    # --- 5. Rampa ---
    sys_ramp = signal.TransferFunction(num, np.polymul(den, [1, 0]))  # H(s) * 1/s
    t_ramp, y_ramp = signal.step(sys_ramp, T=t)
    axs[2, 0].plot(t_ramp, y_ramp, color=color)
    axs[2, 0].set_title("Respuesta a la rampa")
    axs[2, 0].set_xlabel("Tiempo (s)")
    axs[2, 0].grid(True)

    # --- 6. Escalón lazo cerrado ---
    t_cl, y_cl = signal.step(sys_cl, T=t)
    axs[2, 1].plot(t_cl, y_cl, color=color)
    axs[2, 1].set_title("Escalón - Lazo Cerrado")
    axs[2, 1].set_xlabel("Tiempo (s)")
    axs[2, 1].grid(True)

    plt.tight_layout()
    plt.show()

    # --- Mostramos parámetros ---
    print("\n==============================")
    print(f"  {name}")
    print("==============================")
    print(f"m = {m}, c = {c}, k = {k}")
    print(f"ωn = {wn:.4f}  rad/s")
    print(f"ζ = {zeta:.4f}")
    if zeta < 1:
        print(f"ωd = {wd:.4f} rad/s")
        print(f"tp = {tp:.4f} s")
        print(f"tr ≈ {tr:.4f} s")
    print(f"ts = {ts:.4f} s")


# ====================================================
#         CASOS DINÁMICOS OFICIALES
# ====================================================
casos = {
    "Subamortiguado":   {"m": 1.0, "c": 4.0,  "k": 100.0, "color": "blue"},   # ζ = 0.2
    "Crítico":          {"m": 1.0, "c": 20.0, "k": 100.0, "color": "green"},  # ζ = 1
    "Sobreamortiguado": {"m": 1.0, "c": 40.0, "k": 100.0, "color": "red"}     # ζ = 2
}

# ---- Ejecutar todos los casos ----
for nombre, p in casos.items():
    plot_case(nombre, p["m"], p["c"], p["k"], p["color"])

STREAMLIT


In [ ]:
#instalación de librerías
!pip install streamlit -q

In [ ]:
!mkdir pages

In [ ]:
%%writefile 0_👋_Hello.py

import streamlit as st

st.set_page_config(
    page_title="Bienvenida",
    page_icon="👋",
)

st.write("# Bienvenido a Streamlit! 👋")

st.sidebar.success("Seleccciona una demo a explorar.")

st.markdown(
    """
    Streamlit es una aplicación de código abierto creado específicamente para
    Proyectos de Machine Learning y Data Science.
    **👈 Seleccione una demostración de la barra lateral** para ver algunos ejemplos
    ¡De lo que Streamlit puede hacer!
    ### ¿Quieres saber más?
    - Consulta [streamlit.io] (https://streamlit.io)
    - Revisa la [documentación](https://docs.streamlit.io)
"""
)

In [ ]:
PUNTO 2 STREAMLINT

In [ ]:
%%writefile pages/1_⚙️_Masa_Resorte_Amortiguador.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

st.set_page_config(page_title="Sistema Masa-Resorte-Amortiguador", page_icon="⚙️")

st.title("⚙️ Sistema Masa–Resorte–Amortiguador")
st.sidebar.header("Configuración del sistema")

# Entrada de parámetros
m = st.sidebar.number_input("Masa (m)", value=1.0, step=0.1)
c = st.sidebar.number_input("Amortiguamiento (c)", value=4.0, step=0.5)
k = st.sidebar.number_input("Rigidez (k)", value=100.0, step=1.0)

# ====================================================
#   FUNCIÓN PARA GRAFICAR EL SISTEMA
# ====================================================
def plot_system(m, c, k):
    num = [1]
    den = [m, c, k]

    sys = signal.TransferFunction(num, den)
    num_cl = num
    den_cl = np.polyadd(den, num)
    sys_cl = signal.TransferFunction(num_cl, den_cl)

    wn = np.sqrt(k/m)
    zeta = c / (2*np.sqrt(m*k))

    if zeta < 1:
        wd = wn*np.sqrt(1 - zeta**2)
        tp = np.pi / wd
        ts = 4 / (zeta * wn)
        tr = 1.8 / wn
    else:
        wd = None
        tp = None
        tr = None
        ts = 4 / (zeta * wn)

    t = np.linspace(0, 10, 1000)

    fig, axs = plt.subplots(3, 2, figsize=(12, 14))

    # Polos
    z, p, k_tf = signal.tf2zpk(num, den)
    axs[0, 0].scatter(np.real(p), np.imag(p), c="blue", s=60)
    axs[0, 0].axhline(0, color='black')
    axs[0, 0].axvline(0, color='black')
    axs[0, 0].set_title("Polos del Sistema")
    axs[0, 0].grid()

    # Bode magnitud
    w, mag, phase = signal.bode(sys)
    axs[0, 1].semilogx(w, mag)
    axs[0, 1].set_title("Diagrama de Bode - Magnitud")
    axs[0, 1].grid()

    # Impulso
    t_imp, y_imp = signal.impulse(sys, T=t)
    axs[1, 0].plot(t_imp, y_imp)
    axs[1, 0].set_title("Respuesta al Impulso")
    axs[1, 0].grid()

    # Escalón
    t_step, y_step = signal.step(sys, T=t)
    axs[1, 1].plot(t_step, y_step)
    axs[1, 1].set_title("Respuesta al Escalón")
    axs[1, 1].grid()

    # Rampa
    sys_ramp = signal.TransferFunction(num, np.polymul(den, [1, 0]))
    t_ramp, y_ramp = signal.step(sys_ramp, T=t)
    axs[2, 0].plot(t_ramp, y_ramp)
    axs[2, 0].set_title("Respuesta a la Rampa")
    axs[2, 0].grid()

    # Lazo cerrado
    t_cl, y_cl = signal.step(sys_cl, T=t)
    axs[2, 1].plot(t_cl, y_cl)
    axs[2, 1].set_title("Lazo Cerrado - Escalón")
    axs[2, 1].grid()

    plt.tight_layout()
    return fig, wn, zeta, wd, tp, tr, ts

# ----------------------------------------------------
#     BOTÓN PARA GENERAR RESULTADOS Y GRÁFICAS
# ----------------------------------------------------
if st.sidebar.button("Generar Gráficas"):
    with st.spinner("Calculando dinámicas del sistema..."):
        fig, wn, zeta, wd, tp, tr, ts = plot_system(m, c, k)
        st.pyplot(fig)

        st.subheader("📌 Parámetros dinámicos calculados")

        st.write(f"**Frecuencia natural (ωn):** {wn:.4f} rad/s")
        st.write(f"**Coeficiente de amortiguamiento (ζ):** {zeta:.4f}")

        if zeta < 1:
            st.write(f"**Frecuencia amortiguada (ωd):** {wd:.4f} rad/s")
            st.write(f"**Tiempo al pico (tp):** {tp:.4f} s")
            st.write(f"**Tiempo de subida (tr):** {tr:.4f} s")

        st.write(f"**Tiempo de establecimiento (ts):** {ts:.4f} s")
else:
    st.info("Configura los valores en la barra lateral y presiona **Generar Gráficas**.")

In [ ]:
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
from pydub import AudioSegment
import yt_dlp
import os

st.set_page_config(page_title="Modulación y Demodulación AM", page_icon="🔊")

st.title("🔊 Modulación y Demodulación DSB-SC")
st.sidebar.header("Parámetros del Sistema AM")

# ------------------------------------------------------------
# Función: Descargar audio desde YouTube
# ------------------------------------------------------------
def download_audio(url, out_file="message.wav", duration=5):
    st.info("Descargando audio de YouTube...")

    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": "temp_audio.%(ext)s",
        "quiet": True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    # Buscar archivo recién descargado
    downloaded = None
    for f in os.listdir():
        if f.startswith("temp_audio"):
            downloaded = f
            break

    # Convertir a WAV + recortar
    audio = AudioSegment.from_file(downloaded)
    audio = audio.set_channels(1)
    audio = audio.set_frame_rate(44100)
    audio = audio[:duration * 1000]

    audio.export(out_file, format="wav")
    os.remove(downloaded)
    return out_file


# ------------------------------------------------------------
# Utilidades FFT
# ------------------------------------------------------------
def fft_mag(x, fs):
    N = len(x)
    X = np.fft.fftshift(np.fft.fft(x))
    f = np.fft.fftshift(np.fft.fftfreq(N, 1/fs))
    return f, np.abs(X) / N


def apply_lpf_fft(x, fs, fcut):
    N = len(x)
    X = np.fft.fft(x)
    freqs = np.fft.fftfreq(N, 1/fs)
    mask = np.abs(freqs) <= fcut
    return np.real(np.fft.ifft(X * mask))


# ------------------------------------------------------------
# Simulación AM
# ------------------------------------------------------------
def simulate_dsb_demod(audio_file, A1, f0, fcut):
    m, fs = sf.read(audio_file)
    if m.ndim > 1:
        m = m[:, 0]

    N = len(m)
    t = np.arange(N) / fs

    carrier = np.cos(2*np.pi*f0*t)

    # Modulación
    x = A1 * m * carrier

    # Mezclador
    lo = np.cos(2*np.pi*f0*t)
    y = x * lo

    # LPF
    y_lpf = apply_lpf_fft(y, fs, fcut)

    # Recuperación
    recovered = (2/A1) * y_lpf

    # FFTs
    f_m, Mmag = fft_mag(m, fs)
    f_x, Xmag = fft_mag(x, fs)
    f_y, Ymag = fft_mag(y, fs)
    f_yl, Ylmag = fft_mag(y_lpf, fs)
    f_r, Rmag = fft_mag(recovered, fs)

    return (t, m, x, y, y_lpf, recovered,
            (f_m, Mmag), (f_x, Xmag), (f_y, Ymag),
            (f_yl, Ylmag), (f_r, Rmag))


# ------------------------------------------------------------
# Sidebar: Controles del sistema
# ------------------------------------------------------------
A1 = st.sidebar.slider("Amplitud del modulador A1", 0.1, 5.0, 1.0, 0.1)
f0 = st.sidebar.slider("Frecuencia de portadora f0 (Hz)", 1000, 20000, 15000, 500)
fcut = st.sidebar.slider("Frecuencia de corte LPF (Hz)", 100, f0, 5000, 100)

st.sidebar.markdown("---")
st.sidebar.markdown("### Fuente de audio")

option = st.sidebar.radio("Seleccionar método:", ["Subir archivo", "YouTube"])

uploaded_file = None
youtube_url = None

if option == "Subir archivo":
    uploaded_file = st.sidebar.file_uploader("Sube archivo WAV", type=["wav"])
else:
    youtube_url = st.sidebar.text_input("Pega un link de YouTube:")

# ------------------------------------------------------------
# Ejecución principal
# ------------------------------------------------------------
if st.sidebar.button("Procesar"):
    if option == "Subir archivo" and uploaded_file is None:
        st.error("Debes subir un archivo WAV.")
        st.stop()

    if option == "YouTube" and youtube_url.strip() == "":
        st.error("Debes ingresar un enlace de YouTube.")
        st.stop()

    # Obtener archivo WAV
    if option == "Subir archivo":
        with open("input.wav", "wb") as f:
            f.write(uploaded_file.getbuffer())
        audio_file = "input.wav"
    else:
        audio_file = download_audio(youtube_url)

    st.success("Audio listo. Iniciando modulación/demodulación...")

    # Simulación
    t, m, x, y, y_lpf, recovered, \
        M, X, Y, YL, R = simulate_dsb_demod(audio_file, A1, f0, fcut)

    # --------------------------------------------------------
    # Gráficas en tiempo
    # --------------------------------------------------------
    st.header("📈 Gráficas en el Tiempo")

    fig, axs = plt.subplots(5, 1, figsize=(12, 12))
    axs[0].plot(t, m); axs[0].set_title("Mensaje m(t)")
    axs[1].plot(t, x); axs[1].set_title("Señal modulada x(t)")
    axs[2].plot(t, y); axs[2].set_title("Salida del mezclador y(t)")
    axs[3].plot(t, y_lpf); axs[3].set_title("LPF (FFT)")
    axs[4].plot(t, recovered); axs[4].set_title("Señal recuperada m_rec(t)")
    plt.tight_layout()
    st.pyplot(fig)

    # --------------------------------------------------------
    # FFTs
    # --------------------------------------------------------
    st.header("🔍 Espectros de Frecuencia")

    fig2, axs2 = plt.subplots(5, 1, figsize=(12, 12))
    axs2[0].plot(M[0], M[1]); axs2[0].set_title("M(f)")
    axs2[1].plot(X[0], X[1]); axs2[1].set_title("X(f)")
    axs2[2].plot(Y[0], Y[1]); axs2[2].set_title("Y(f)")
    axs2[3].plot(YL[0], YL[1]); axs2[3].set_title("Y_LPF(f)")
    axs2[4].plot(R[0], R[1]); axs2[4].set_title("M_rec(f)")
    plt.tight_layout()
    st.pyplot(fig2)

else:
    st.info("Configura parámetros y presiona **Procesar**.")


In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run 0_👋_Hello.py &>/content/logs.txt & #Cambiar 0_👋_Hello.py por el nombre de tu archivo principal

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL

FINALIZACION DE EJECUCION DASHBOARD

In [ ]:
import os

res = input("Digite (1) para finalizar la ejecución del Dashboard: ")

if res.upper() == "1":
    os.system("pkill streamlit")  # Termina el proceso de Streamlit
    print("El proceso de Streamlit ha sido finalizado.")